In [34]:
import matplotlib.pyplot as plt
import cv2 
import glob
import tarfile 
import numpy as np
import os

from skimage.io import imread, imshow
%matplotlib inline

In [35]:
with tarfile.open('./data/PedCut2013_SegmentationDataset.tar.gz') as tar:
  tar.extractall()

img_size = 256
input_path = glob.glob('./data/testData/left_images/*')
output_path = glob.glob('./data/testData/left_groundTruth/*')

img_num = len(input_path)

dir = os.listdir('./data/')
print(dir)
# print(input_path, output_path)
# print(output_path)

['testData', '.DS_Store', 'validationData', 'PedCut2013_SegmentationDataset.tar.gz', 'completeData']
['./data/testData/left_groundTruth/nr_155_img_000000042_c0.png', './data/testData/left_groundTruth/nr_775_img_000000224_c0.png', './data/testData/left_groundTruth/nr_076_img_000000022_c0.png', './data/testData/left_groundTruth/nr_169_img_000000044_c0.png', './data/testData/left_groundTruth/nr_563_img_000000137_c0.png', './data/testData/left_groundTruth/nr_730_img_000000195_c0.png', './data/testData/left_groundTruth/nr_228_img_000000058_c0.png', './data/testData/left_groundTruth/nr_375_img_000000101_c0.png', './data/testData/left_groundTruth/nr_384_img_000000102_c0.png', './data/testData/left_groundTruth/nr_763_img_000000216_c0.png', './data/testData/left_groundTruth/nr_571_img_000000139_c0.png', './data/testData/left_groundTruth/nr_665_img_000000165_c0.png', './data/testData/left_groundTruth/nr_351_img_000000095_c0.png', './data/testData/left_groundTruth/nr_365_img_000000100_c0.png', '.

In [36]:
## データを数値に変換
x_train = []
y_train = []

for path in input_path:
  image = cv2.imread(path)/255.
  image = cv2.resize(image, (img_size, img_size))
  x_train.append(image)

for path in output_path:
  image = cv2.imread(path, 0)/255.
  image = cv2.resize(image, (img_size, img_size))
  y_train.append(image)

x_train = np.array(x_train)
y_train = np.array(y_train)

In [37]:
print(len(x_train), len(y_train))
# print(x_train, y_train)
print(type(x_train), type(y_train))

300 300
[[[[0.32084865 0.47094822 0.51016391]
   [0.32801837 0.48411488 0.52100582]
   [0.33787683 0.50641085 0.54219363]
   ...
   [0.28562045 0.39920959 0.40445162]
   [0.25592065 0.3553848  0.35141727]
   [0.24101563 0.33157169 0.31908701]]

  [[0.31372549 0.47558977 0.51372549]
   [0.3292169  0.49086492 0.52650123]
   [0.34718595 0.51545033 0.5482108 ]
   ...
   [0.30350348 0.43592843 0.43024355]
   [0.27882041 0.39129923 0.37561026]
   [0.26098346 0.35834099 0.33765319]]

  [[0.31552543 0.4748315  0.51372549]
   [0.33158243 0.48840987 0.52650123]
   [0.35070466 0.51206348 0.5475337 ]
   ...
   [0.32653186 0.48160209 0.45844053]
   [0.29859526 0.42577051 0.39880515]
   [0.27206648 0.37794884 0.35621936]]

  ...

  [[0.39607843 0.47875306 0.43953738]
   [0.39492645 0.47760108 0.43953738]
   [0.39547344 0.47620254 0.43988962]
   ...
   [0.3372549  0.40816483 0.36333484]
   [0.3372549  0.40898746 0.36372857]
   [0.3372549  0.40996477 0.36470588]]

  [[0.40176164 0.48235294 0.44313725]

In [38]:
import eval_index as ev

In [39]:
import unetLib as unet
unet_model = unet.build_model()

<class 'tensorflow.python.framework.ops.Tensor'>
<class 'filter'>


In [41]:
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.utils import plot_model


unet_model.compile(loss=ev.dice_coef_loss, optimizer=RMSprop(lr=1e-4), metrics=[ev.dice_coef])
unet_model.summary()
plot_model(unet_model, show_shapes=True, to_file='./arch/unetarch.png')

batch_size=8
epoch_num=100

history = unet_model.fit(x_train, y_train, batch_size=batch_size, epochs=epoch_num)

# unet_model.save('u-net.hdf5') # モデルの保存
pred = unet_model.predict(x_train)

for i, y in enumerate(pred[:6]):
    print("train_img")
    imshow(x_train[i])
    plt.show()
    print("pred")
    imshow(pred[i].reshape(256, 256))
    plt.show()


Train on 300 samples
Epoch 1/100
 24/300 [=>............................] - ETA: 30:40 - loss: 0.5488 - dice_coef: 0.4512